# Expirements for Task 2

According to Task 2, I will conduct the follwings expirements:
- (1) Compare Skip-gram, Skip-gram negative sampling, GloVe models on training loss, training time. 
- (2) Use Word analogies dataset to calucalte between syntactic and semantic accuracy, similar to the
methods in the Word2Vec and GloVe paper. 
- (3) Use the similarity dataset to find the correlation between your models’ dot product and the provided similarity metrics. (from scipy.stats import spearmanr) Assess if your embeddings correlate with human judgment.

## Importing

In [12]:
# import 3 models from main.py
from main import Skipgram, SkipgramNeg, Glove

In [6]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [7]:
import torch
import torch.nn.functional as F
import pickle
import numpy as np
from scipy.stats import spearmanr

In [8]:
# Importing training data
Data = pickle.load(open('./model/Data.pkl', 'rb'))
corpus = Data['corpus']
vocab = Data['vocab']
word2index = Data['word2index']
voc_size = Data['voc_size']
embed_size = Data['embedding_size']

## Load the Model

In [16]:
# create object of the Skipgram model and load parameters
Skipgram = Skipgram(voc_size, embed_size)
Skipgram.load_state_dict(torch.load('model/A1-Skipgram.pt'))
Skipgram.eval()

Skipgram(
  (embedding_center): Embedding(4167, 2)
  (embedding_outside): Embedding(4167, 2)
)

In [ ]:
# create object of the SkipgramNeg model and load parameters
SkipgramNeg = SkipgramNeg(voc_size, embed_size)
SkipgramNeg.load_state_dict(torch.load('model/A1-SkipgramNeg.pt'))
SkipgramNeg.eval()

In [ ]:
# # Instantiate the model and load saved parameters
# glove = Glove(voc_size, embed_size)
# glove.load_state_dict(torch.load('app/models/GloVe-v1.pt'))
# glove.eval()

In [9]:
# you have to put this file in some python/gensim directory; just run it and it will inform where to put
glove_file = ('./glove.6B/glove.6B.100d.txt')
model = KeyedVectors.load_word2vec_format(glove_file, binary = False, no_header = True)

## Function for semantic and syntactic analysis 
that calculates similarities using cosine similarity

In [10]:
import torch
import torch.nn.functional as F

def calculate_similarity(word_vectors, result_vector):
    # Calculate cosine similarities
    similarities = F.cosine_similarity(result_vector, word_vectors)

    # Find the index of the word with the highest similarity
    closest_word_index = torch.argmax(similarities).item()

    return closest_word_index

def similarities(lines, model, vocab):
    # Get word vectors for all words in the vocabulary
    all_word_vectors = torch.stack([model.get_embed(word.lower()) for word in vocab])

    correct = 0
    # Perform vector manipulation for each set of four words
    for line in lines:
        words = line.split()

        # Assuming there are four words in each line
        vectors = [model.get_embed(word.lower()) if word in vocab else model.get_embed('<UNK>') for word in words]

        # Perform vector manipulation (e.g., subtraction, addition)
        result_vector = vectors[1][0] - vectors[0][0] + vectors[2][0]

        # Add a batch dimension to result_vector
        result_vector = result_vector.unsqueeze(0)

        # Find the closest word index using cosine similarity
        closest_word_index = calculate_similarity(all_word_vectors, result_vector)

        # Get the closest word from your vocabulary
        closest_word = vocab[closest_word_index]

        if closest_word == words[3]:
            correct += 1

        # Optionally, you can print the result for each line
        # print(f"The word with the closest cosine similarity to the result of {line} is: {closest_word}")

    print('---------------------------------------------------------')
    print(f'Accuracy : {(correct / len(lines)) * 100: .2f}%')
    return


## Testing Phase

### Load the Data

### (1) syntactic and semantic accuracy

### (2) Similarity

### Human Model